# Push data from log_action.csv to kafka topic vdt2024 using python

In [1]:
%pip install confluent_kafka
%pip install kafka-python

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from confluent_kafka import Producer
import socket
import time
import io

import json


producer = Producer({

    'bootstrap.servers': 'broker01:9092',
    'client.id': socket.gethostname(),

    "batch.size": 1,

})

topic = 'vdt2024'



def acked(err, msg):

    if err is not None:

        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:

        print("Message produced: %s" % (str(msg)))



with open('./data/log_action.csv') as csv_file:

    idx = 0

    for line in csv_file:

        fields = line.replace('\n', '').split(',')

        data_set = {

            "student_code": int(fields[0]),

            "activity": fields[1],

            "numberOfFile": int(fields[2]),

            "timestamp": fields[3]

        }

        producer.produce(topic, key=str(idx), value=json.dumps(data_set), callback=acked)
        print('message sent')

        producer.flush()
        print('message flush')
        break


        time.sleep(1)

message sent
Failed to deliver message: <cimpl.Message object at 0x7f23f5c616c0>: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
message flush


## Initialize spark session

In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext('local')
spark = SparkSession(sc)

## Create data schema

In [5]:
from pyspark.sql.types import StructType

action_schema = StructType().add("student_code", "integer")\
    .add("activity", "string")\
    .add("numberOfFile", "integer")\
    .add("timestamp", "string")

sv_de_schema = StructType().add("student_code", "integer")\
    .add("fullname", "string")

## Save danh_sach_sv_de to hdfs with path: /raw_zone/danh_sach_sv_de.csv

In [3]:
df = spark.read.format("csv").option("header", "false").schema(sv_de_schema)\
    .load("./data/danh_sach_sv_de.csv")
df.show()

+------------+--------------------+
|student_code|            fullname|
+------------+--------------------+
|           1|          Mai Đức An|
|           2|      Nguyễn Mai Anh|
|           3|   Ngô Ngọc Tuấn Anh|
|           4|      Trần Trung Anh|
|           5|       Trần Ngọc Bảo|
|           6|  Nguyễn Vũ Hòa Bình|
|           7|    Nguyễn Thành Đạt|
|           8|        Đỗ Thành Đạt|
|           9|    Nguyễn Khoa Đoàn|
|          10|    Nguyễn Quốc Dũng|
|          11|     Đường Minh Quân|
|          12|   Dương Quang Giang|
|          13|    Nguyễn Minh Hiếu|
|          14|        Ngô Phi Hùng|
|          15|Nguyễn Đình Thiên...|
|          16|        Đỗ Doãn Khắc|
|          17|      Châu Minh Khải|
|          18|      Phạm Đình Khôi|
|          19|        Lê Bảo Khánh|
|          20|        Lê Minh Phúc|
+------------+--------------------+
only showing top 20 rows



In [4]:
df.write.format("csv").option("header", "false").mode("overwrite")\
    .save("hdfs://namenode:9000/raw_zone/danh_sach_sv_de")

### check data in hdfs

In [7]:
sv_de_df = spark.read.format("csv").option("header", "false").schema(sv_de_schema)\
    .load("hdfs://namenode:9000/raw_zone/danh_sach_sv_de")
sv_de_df.show(40)

AnalysisException: Path does not exist: hdfs://namenode:9000/raw_zone/danh_sach_sv_de

In [ ]:
action_data = spark.read.csv("hdfs://namenode:9000/user/hadoop/data/action_data.csv",
                             header=True, schema=action_schema)